In [ ]:
import os
import pandas as pd
import json
import folium
import copy

In [ ]:
# load the data regarding the ratio of unemployement
europe_unemploy_rate_path = r'topojson/lfsa_urgan_1_Data.csv'
europe_unemploy_rate = pd.read_csv(europe_unemploy_rate_path)
europe_unemploy_rate = europe_unemploy_rate.loc[\
    (europe_unemploy_rate.TIME==2016) & (europe_unemploy_rate.SEX=="Total") & (~europe_unemploy_rate.GEO.str.contains("European|Euro")), \
    ["GEO","Value"]]
europe_unemploy_rate.loc[europe_unemploy_rate.GEO.str.contains("Germany"), "GEO"] = "Germany"
europe_unemploy_rate.loc[europe_unemploy_rate.GEO.str.contains("Former Yugoslav Republic of Macedonia"), "GEO"] = "The former Yugoslav Republic of Macedonia"
print("We have the unemployement rate the following states:")
europe_unemploy_rate.shape[0], list(europe_unemploy_rate.GEO)

In [ ]:
# load the europe data (geometry of the states) 
europe_topo_path = r'topojson/europe.topojson.json'
topo_data = json.load(open(europe_topo_path))
topo_states = [state["properties"]["NAME"] for state in topo_data["objects"]["europe"]["geometries"]]
print("We draw the boundaries of the following states:")
len(topo_states), topo_states

In [ ]:
# we don't have the data of all the states!
missing_states = list(set(topo_states)-set(europe_unemploy_rate.GEO))
print("We are missing the unemployement rate about these states:")
len(missing_states), missing_states

In [ ]:
# append the missing states with 0, we will treat them later
missing_states = pd.DataFrame({"GEO":missing_states, 
             "Value": pd.Series([0]*len(missing_states))})

In [ ]:
europe_unemploy_rate = europe_unemploy_rate.append(missing_states).reset_index().loc[:, ["GEO", "Value"]]

In [ ]:
europe_unemploy_rate

In [ ]:
def fill_invalid(state):
    if state in list(missing_states.GEO):
        return 0.7
    else:
        return 0

In [ ]:
europe_location = [55, 15]

m = folium.Map(location=europe_location, zoom_start=3)

# fill the states with colors depending on their unemployement rate
m.choropleth(
    geo_data=copy.deepcopy(topo_data), 
    topojson='objects.europe',
    data=europe_unemploy_rate,
    columns=['GEO', 'Value'],
    key_on='properties.NAME',
    fill_color='BuGn', 
    fill_opacity=0.7, 
    #line_opacity=1,
    legend_name='Percentage of unemployement (%)')

# draw better boundaries
folium.TopoJson(
    topo_data,
    'objects.europe',
    name='topojson',
    style_function=lambda geometry: {
#         # boundaries
#         'color' : 'black',
#         'weight' : 1,
        # fill invalid states with grey
        'fillColor': "#424949", 
        'fillOpacity': fill_invalid(geometry["properties"]["NAME"]), 
        }
).add_to(m)

m